</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [3]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True)
node(input_dict)

KeyboardInterrupt: 


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [ ]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)


MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [ ]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [ ]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### EvalNode Test: 
At the beginning level, generation of EvalNode (when code is used here) should compile with success, and it should takes input and produce output. 

In [1]:
# Usage
from tools.repo import *

repo_url = "https://github.com/xjdr-alt/entropix.git"

temp_repo = "temp_repo_1"
if not os.path.exists(temp_repo):
    clone_repo(repo_url, temp_repo)
else:
    print("Repo already cloned, skipping cloning...")
    
dates, dags = obtain_repo_evolution(temp_repo)

Repo already cloned, skipping cloning...


In [2]:
from tools.diagram import *
dag = dags[0]

# TBD: loop through all commit dag struture, and animate the growing process

dag = assign_importance_score_to_dag(dag)

sub_dag = decide_opacity_of_dag(dag, progress=1.0, cap_node_number=15)

d2_code = build_d2_from_dag(sub_dag, include_overhead=True)

# Write a function to store d2_code into .d2 file under local directory, also store .svg file
save_d2_and_svg(d2_code, "test_evo")

D2 diagram saved as d2_output/test_evo.d2
SVG file generated at d2_output/test_evo.svg


success: successfully compiled d2_output/test_evo.d2 to d2_output/test_evo.svg in 308.887ms


('d2_output/test_evo.d2', 'd2_output/test_evo.svg')

In [1]:
from tools.diagram import *
import time

# # TBD: loop through all commit dag struture, and animate the growing process
# for i, dag in enumerate(dags):
#     dag = assign_importance_score_to_dag(dag)
    
#     # Animate the growing process by increasing progress from 0 to 1
#     for progress in np.linspace(0, 1, 10):
#         sub_dag = decide_opacity_of_dag(dag, progress=progress, cap_node_number=15)
#         d2_code = build_d2_from_dag(sub_dag, include_overhead=True)
        
#         # Save each frame as an SVG
#         filename = f"commit_{i}_progress_{progress:.2f}"
#         save_d2_and_svg(d2_code, filename)
        
#         time.sleep(0.5)  # Add a small delay between frames

# After generating all SVGs, combine them into a GIF
from PIL import Image
import glob
import os

# Get all d2 files
d2_files = sorted(glob.glob("d2_output/*.d2"))

# Convert D2 to PNGs
png_files = []
for d2_file in d2_files:
    png_file = d2_file.replace(".d2", ".png")
    save_d2_as_png(d2_file, png_file)
    png_files.append(png_file)

# Create GIF
create_gif(png_files)

success: successfully compiled d2_output/commit_0_progress_0.00.d2 to d2_output/commit_0_progress_0.00.png in 192.676916ms


D2 diagram saved as d2_output/commit_0_progress_0.00.png


success: successfully compiled d2_output/commit_0_progress_0.11.d2 to d2_output/commit_0_progress_0.11.png in 747.872667ms


D2 diagram saved as d2_output/commit_0_progress_0.11.png


success: successfully compiled d2_output/commit_0_progress_0.22.d2 to d2_output/commit_0_progress_0.22.png in 748.466666ms


...

D2 diagram saved as d2_output/commit_6_progress_0.78.png


success: successfully compiled d2_output/commit_6_progress_0.89.d2 to d2_output/commit_6_progress_0.89.png in 749.097334ms


D2 diagram saved as d2_output/commit_6_progress_0.89.png


success: successfully compiled d2_output/commit_6_progress_1.00.d2 to d2_output/commit_6_progress_1.00.png in 745.666792ms


D2 diagram saved as d2_output/commit_6_progress_1.00.png
Animation saved as commit_dag_evolution.gif


In [ ]:
from tools.repo import *

repo_url = "https://github.com/yilundu/ired_code_release"

temp_repo = "temp_repo"
if not os.path.exists(temp_repo):
    clone_repo(repo_url, temp_repo)
else:
    print("Repo already cloned, skipping cloning...")

python_files = get_python_files(temp_repo) # get all python files 
start_file = python_files[0] # pick the first one for debugging purpose 

dag = build_cross_file_dag(temp_repo, start_file)

Repo already cloned, skipping cloning...


In [ ]:
# Raw material for sub-graph (mermaid) building & saving

target_node = "models.py"
# target_node = "Attend"
max_depth = 6

name_map = {dag[k]["name"]: k for k in dag} # Map name to node-id for all nodes in the DAG

center_node = name_map[target_node] # Node id or target_node

sub_dag = extract_subgraph_dag(dag, center_node, max_depth)


In [ ]:
print(f"Number of keys in sub_dag: {len(sub_dag.keys())}")

Number of keys in sub_dag: 110


In [ ]:
from tools.diagram import build_d2_from_dag, save_d2_and_svg, decide_opacity_of_dag, filter_opacity_graph

progress = 1.0
sub_dag = decide_opacity_of_dag(sub_dag, progress, cap_node_number=20)

d2_code = build_d2_from_dag(sub_dag, include_overhead=False)

# Write a function to store d2_code into .d2 file under local directory, also store .svg file
save_d2_and_svg(d2_code, "test")

D2 diagram saved as d2_output/test.d2
SVG file generated at d2_output/test.svg


success: successfully compiled d2_output/test.d2 to d2_output/test.svg in 199.742291ms


('d2_output/test.d2', 'd2_output/test.svg')

#### Toy-Case illustrating desired behavior: 
* Animating Dependency Graph Evolution 

In [ ]:
import os
from PIL import Image
import subprocess

def create_growing_graph_animation(mermaid_code, output_file="growing_graph.gif"):
    # Split the Mermaid code into stages
    stages = mermaid_code.strip().split("%% Stage")[1:]# Skip the first empty split
    stages = [s.strip() for s in stages]
    
    png_files = []
    codes = []
    
    # Generate PNG for each stage
    for i, stage in enumerate(stages, 1):
        # Combine all previous stages with the current one
        combined_stage = "graph TD\n" + "\n".join(stages[:i]).strip()
        
        with open(f"stage_{i}.mmd", "w") as f:
            f.write(combined_stage)
        
        # Use mmdc to generate PNG directly
        subprocess.run(["mmdc", "-i", f"stage_{i}.mmd", "-o", f"stage_{i}.png"], check=True)
        png_files.append(f"stage_{i}.png")
        
        print(f"Stage {i} code:")
        print(combined_stage)
        codes.append(combined_stage)
        print()  # Add a blank line for readability
    
    # Create animated GIF
    images = [Image.open(f) for f in png_files]
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=1000, loop=0)
    
    # Clean up temporary files
    # for file in png_files + [f"stage_{i}.mmd" for i in range(1, len(stages) + 1)]:
    #     os.remove(file)
    
    print(f"Animation saved as {output_file}")
    
    return codes

# Usage
mermaid_code = """
%% Stage
%%{init: {'theme': 'base', 'themeVariables': { 'fontSize': '16px'}, 'flowchart': {'width': 800, 'height': 600}}}%%
A[Start, width=80px, height=40px, style=filled, color=lightgreen, stroke-width=2px]
style A fill:#f9f,stroke:#333,stroke-width:2px

%% Stage
B[Second, width=80px, height=40px, style=filled, color=lightblue, stroke-width=2px]
style B fill:#f9f,stroke:#333,stroke-width:2px
A --> B

%% Stage
C[Third, width=80px, height=40px, style=filled, color=lightyellow, stroke-width=2px]
style C fill:#f9f,stroke:#333,stroke-width:2px
B --> C

%% Stage
D[Fourth, width=80px, height=40px, style=filled, color=lightpink, stroke-width=2px]
style D fill:#f9f,stroke:#333,stroke-width:2px
C --> D
"""

code = create_growing_graph_animation(mermaid_code)

Generating single mermaid chart
Stage 1 code:
graph TD
%%{init: {'theme': 'base', 'themeVariables': { 'fontSize': '16px'}, 'flowchart': {'width': 800, 'height': 600}}}%%
A[Start, width=80px, height=40px, style=filled, color=lightgreen, stroke-width=2px]
style A fill:#f9f,stroke:#333,stroke-width:2px

Generating single mermaid chart
Stage 2 code:
graph TD
%%{init: {'theme': 'base', 'themeVariables': { 'fontSize': '16px'}, 'flowchart': {'width': 800, 'height': 600}}}%%
A[Start, width=80px, height=40px, style=filled, color=lightgreen, stroke-width=2px]
style A fill:#f9f,stroke:#333,stroke-width:2px
B[Second, width=80px, height=40px, style=filled, color=lightblue, stroke-width=2px]
style B fill:#f9f,stroke:#333,stroke-width:2px
A --> B

Generating single mermaid chart
Stage 3 code:
graph TD
%%{init: {'theme': 'base', 'themeVariables': { 'fontSize': '16px'}, 'flowchart': {'width': 800, 'height': 600}}}%%
A[Start, width=80px, height=40px, style=filled, color=lightgreen, stroke-width=2px]
sty

KeyboardInterrupt: 

In [ ]:
# BreakDown on Task (When something is hard, we break it down)

# 1. Construct step-wise SVG
#    - transparency changing function
#    - growing strategy (llm aided)

# 2. Get sequence of SVG and animate it into gif

